# **Task 1 : Load Balancing for Package Delivery using Beam Search**

A delivery company has multiple trucks, each capable of carrying a set number of packages. Your goal is to assign packages to trucks in a way that minimizes the total delivery time. Due to the large number of possible assignments, you will use Local Beam Search to efficiently explore the best delivery assignments.

### **Input:**

- A list of N packages, each with a specific delivery time (in minutes).
    - Example: packages = [20, 35, 10, 25, 40, 15, 30, 22]
- A set of M trucks.
    - Example: M = 3
- Beam width (number of best states to keep at each step).
    - Example: Beam Width = 3

### **Output:**
- A schedule showing which package is assigned to which truck.
- The total delivery time for each truck.

#### **Expected Output**
```
Best Package-Truck Assignment:
Truck 1: [20, 35]  --> Total Time: 55 minutes
Truck 2: [10, 40, 15]  --> Total Time: 65 minutes
Truck 3: [25, 30, 22]  --> Total Time: 77 minutes
```


### **Constraints:**
- The goal is to minimize the total delivery time (i.e., minimize the maximum load across all trucks).
- The heuristic function should evaluate states based on makespan ( **Makespan = max(Total time for each truck)** ).
- Keep only the k-best states at each step (k = beam width).


### **Algorithm Steps**

1. **Initialize Parameters**  
   - Read **N packages**, **M trucks**, and **Beam Width (k)**.  
   - Start with `k` initial states by assigning the first few packages in different ways.  

2. **Iterate Until All Packages Are Assigned**  
   - For each state in the beam, generate new states by assigning the next package to any truck.  
   - Compute **makespan** for each state:  
     ```
     Makespan = max(total delivery time of any truck)
     ```
   - Keep only the `k` best states (lowest makespan).  

3. **Terminate & Output Best Assignment**  
   - Once all packages are assigned, return the state with the **minimum makespan**.  
   - Output **truck assignments** and **total delivery time per truck**.  



In [5]:
# Adjust the code accordingly

def calculate_makespan(assignments):

    # delivery time for truck takimg longest.
    truck_totals = []
    for truck in assignments:
      truck_totals.append(sum(truck))  #total time for each truck
    return max(truck_totals)


def get_successor_states(current_states, beam_width,num_trucks,package):

    # current_states -> list of possible truck assigment states
    successors = []
    for state in current_states:
        for i in range(num_trucks):
            new_state = [list(truck) for truck in state]  # deep copy current statess
            new_state[i].append(package)  # Add package to the i-th truck
            successors.append(new_state)

    successors.sort(key=calculate_makespan)  # Sort states by the lowest makespan
    return successors[:beam_width]  # Keep only the top beam_width states


def beam_search(packages, num_trucks, beam_width):

    # list of states with one state (with all empty trucks)
    current_states = [[[] for i in range(num_trucks)]]

    for package in packages:
        current_states = get_successor_states(current_states, beam_width,num_trucks,package)
        print(f"States for package {package}: ",current_states)

    # Select best assignment from final states
    best_assignment = min(current_states, key=calculate_makespan)

    # Print results
    print("\nBest Package-Truck Assignment:")
    for i, truck in enumerate(best_assignment):
        print(f"Truck {i + 1}: {truck}  --> Total Time: {sum(truck)} minutes")


packages = [20, 35, 10, 25, 40, 15, 30, 22]  #delivery times
num_trucks = 3
beam_width = 2

# Running Beam Search
beam_search(packages, num_trucks, beam_width)

States for package 20:  [[[20], [], []], [[], [20], []]]
States for package 35:  [[[20], [35], []], [[20], [], [35]]]
States for package 10:  [[[20, 10], [35], []], [[20], [35], [10]]]
States for package 25:  [[[20, 10], [35], [25]], [[20], [35], [10, 25]]]
States for package 40:  [[[20, 40], [35], [10, 25]], [[20, 10], [35], [25, 40]]]
States for package 15:  [[[20, 40], [35, 15], [10, 25]], [[20, 40], [35], [10, 25, 15]]]
States for package 30:  [[[20, 40], [35, 15], [10, 25, 30]], [[20, 40], [35, 30], [10, 25, 15]]]
States for package 22:  [[[20, 40], [35, 15, 22], [10, 25, 30]], [[20, 40], [35, 30], [10, 25, 15, 22]]]

Best Package-Truck Assignment:
Truck 1: [20, 40]  --> Total Time: 60 minutes
Truck 2: [35, 15, 22]  --> Total Time: 72 minutes
Truck 3: [10, 25, 30]  --> Total Time: 65 minutes


# **Task 2 : Finding the Best Seat in a Movie Theater Using Simulated Annealing**

You are in a crowded movie theater, trying to find the best seat. The goal is to get a seat that balances viewing experience (distance from the screen) and comfort (avoiding noisy neighbors). However, the best seats may not always be available, and you may need to explore different options before settling on one.


Each seat has a comfort score based on:

- Row Distance: How far the seat is from the middle row.
- Column Distance: How far the seat is from the middle column.
- Filled neighboring Seats: The number of occupied seats nearby.

Your goal is to find the best available seat using Simulated Annealing, optimizing for comfort while balancing exploration and exploitation.

### **Objective Function (Seat Score)**
Each seat’s discomfort is calculated as:
    
##### `𝐷 = Row Distance + Column Distance`

A lower score means a better seat.

### **Temprature Decay after each iteration**

T=T-1




## **Algorithm Steps**
1. **Start at a seat (last row , last column).**
2. **Pick a valid (not occupied) neighboring seat randomly** (move up, down, left, or right).
3. **Compare discomfort scores**:
   - If the new seat is **better (lower D)** than the current one, move there.
   - If it’s **worse (higher D)**, accept it with probability:

     $$
     P = e^{-\frac{\Delta D}{T}}
     $$

     where:
     - \( ${\Delta D}$ = Distance of current seat - Distance of new seat \)
     - \( T \) is the current temperature.

   - If the probability \( P \) is **greater than 0.8**, accept the move ( update the current ); otherwise, skip this neighbor and select the next.
4. **Update the best seat found so far** (if the distance of the current is less than best seat found).
5. **Reduce the temperature** after each step.
6. **Stop when**:
   - **Temperature drops below 20.**


In [13]:
import random
import math

# ⬜ -> not allowed
# 💺 -> empty seats
seats = [
    ["⬜", "⬜", "💺", "💺", "💺", "⬜", "⬜"],
    ["⬜", "💺", "💺", "💺", "💺", "💺", "⬜"],
    ["💺", "💺", "💺", "💺", "💺", "💺", "💺"],
    ["💺", "💺", "💺", "💺", "💺", "💺", "💺"],
    ["💺", "💺", "💺", "💺", "💺", "💺", "💺"]
]

# write the code here accordingly
# output best seat so far

theater = []
for row in seats:
    theater.append([0 if seat == "💺" else 1 for seat in row])

# Get the dimensions of the theater
rows = len(theater)
cols = len(theater[0])

def calculate_discomfort(seat):
    # floor division for middle
    mid_row = rows // 2
    mid_col = cols // 2

    #distance from mid
    row_dist = abs(seat[0] - mid_row)
    col_dist = abs(seat[1] - mid_col)

    # Initialize the count of filled neighboring seats
    filled_neighbors = 0

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dr, dc in directions:
      # neighbor index
      neighbor_row = seat[0] + dr
      neighbor_col = seat[1] + dc

      # neighbor in bounds and occupied
      if 0 <= neighbor_row < rows and 0 <= neighbor_col < cols and theater[neighbor_row][neighbor_col] == 1:
        filled_neighbors += 1

    return row_dist + col_dist + filled_neighbors

# Simulated annealing algorithm to find the best seat
def simulated_annealing(initial_temp=100, min_temp=20):
    # Start at the last row and column
    current_seat = (rows - 1, cols - 1)
    best_seat = current_seat
    current_temp = initial_temp

    while current_temp > min_temp:
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in directions:
            neighbor_row = current_seat[0] + dr
            neighbor_col = current_seat[1] + dc
            # empty aand valid
            if 0 <= neighbor_row < rows and 0 <= neighbor_col < cols and theater[neighbor_row][neighbor_col] == 0:
              neighbors.append((neighbor_row, neighbor_col))

        if not neighbors:
            break

        # choose random
        new_seat = random.choice(neighbors)

        curr_discomfort = calculate_discomfort(current_seat)
        new_discomfort = calculate_discomfort(new_seat)

        #accept new if lower discomfort
        if new_discomfort < curr_discomfort:
            current_seat = new_seat
        else:
            # accept with probability on temperature
            delta_D = curr_discomfort - new_discomfort
            if math.exp(-delta_D / current_temp) > 0.8:
                # Accept with high prob
                current_seat = new_seat

        # Update best if new lower
        if calculate_discomfort(current_seat) < calculate_discomfort(best_seat):
            best_seat = current_seat

        current_temp -= 1

    return best_seat, calculate_discomfort(best_seat)


best_seat, best_discomfort = simulated_annealing()
print("Best Seat Found:", best_seat)
print("Discomfort Score:", best_discomfort)


Best Seat Found: (2, 3)
Discomfort Score: 0
